In [1]:
# [Steps to solve the APE task]
# load rowdata and datalive 2 spreadsheets to python memory
# make a copy of datalive - datalive_update because we will update it by operations
# generate a list of policy numbers for per rowdata and datalive
# find new policy numbers, which exist in rawdata but not in datalive 
# remove all empty rows in datalive_update
# add new policy numbers as new rows in datalive_update
# remove all empty dolumns in datalive_update
# populate 2020-05-01 column in datalive_update table by Premium value in rowdata table, and the matching fields are policy number and time
# for those new added rows, populate 2019-10-01 column in datalive_update table by Premium value in rowdata table, and the matching fields are policy number and time equal to 2019-10-01
# do the same for 2019-11-01, 2019-12-01, 2020-01-01, 2020-02-01, 2020-03-01, 2020-04-01
# for each policy in datalive, if payments < 4 then set Freq to 'ASP'
# for each policy in datalive, if payments >= 4 and all payments are the same then set Freq to 'M'
# for each policy in datalive, if payments >=4 but all payments are not the same then mark 'Check SP' as 'Yes'

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
# load rowdata and datalive 2 spreadsheets to python memory
rowdata = pd.read_excel('../sampledata/APESample.xlsx', sheet_name='Raw data')
datalive = pd.read_excel('../sampledata/APESample.xlsx', sheet_name='Data live')

# make a copy of datalive - datalive_update because we will update it by operations
datalive_update = datalive.copy()

In [4]:
# generate a list of policy numbers for per rowdata and datalive
rowdata_policynolist = rowdata['Policy Nr'].values.tolist()
datalive_policynolist = datalive['Policy Number'].values.tolist()

rowdata_policynolist_distinct = list(set(rowdata_policynolist))
datalive_policynolist_distinct = list(set(datalive_policynolist))

# find new policy numbers, which exist in rawdata but not in datalive 
newpolicynolist = np.setdiff1d(rowdata_policynolist_distinct,datalive_policynolist_distinct).tolist()

# remove all empty rows in datalive_update
datalive_update = datalive_update[datalive_update['Policy Number'].isnull() != True]

# add new policy numbers as new rows in datalive_update
for p in newpolicynolist:
    datalive_update = datalive_update.append({'Policy Number': p, 'Freq': 'ASP'}, ignore_index=True)

# remove all empty dolumns in datalive_update
datalive_update = datalive_update.iloc[:, 0:25]

In [5]:
# populate 2020-05-01 column in datalive_update table by Premium value in rowdata table, 
# and the matching fields are policy number and time equal to 2020-05-01
for index in datalive_update.index:    
    policyno = datalive_update.loc[index]['Policy Number']    
    premium = rowdata.loc[(rowdata['Policy Nr'] == policyno) & (rowdata['time'] == '2020-05-01')]['Premium']
    if len(premium.values) > 0:
        datalive_update.iloc[index,11] = premium.values[0]


In [6]:
# for those new added rows, populate 2019-10-01 and other date columns in datalive_update table by Premium value in rowdata table, 
# and the matching fields are policy number and time
for policyno in newpolicynolist:
    premium = rowdata.loc[(rowdata['Policy Nr'] == policyno) & (rowdata['time'] == '2019-10-01')]['Premium']
    if len(premium.values) > 0:
        datalive_update.loc[datalive_update['Policy Number'] == policyno, 4] = premium.values[0]
        
    premium = rowdata.loc[(rowdata['Policy Nr'] == policyno) & (rowdata['time'] == '2019-11-01')]['Premium']
    if len(premium.values) > 0:
        datalive_update.loc[datalive_update['Policy Number'] == policyno, 5] = premium.values[0]
        
    premium = rowdata.loc[(rowdata['Policy Nr'] == policyno) & (rowdata['time'] == '2019-12-01')]['Premium']
    if len(premium.values) > 0:
        datalive_update.loc[datalive_update['Policy Number'] == policyno, 6] = premium.values[0]
        
    premium = rowdata.loc[(rowdata['Policy Nr'] == policyno) & (rowdata['time'] == '2020-01-01')]['Premium']
    if len(premium.values) > 0:
        datalive_update.loc[datalive_update['Policy Number'] == policyno, 7] = premium.values[0]
        
    premium = rowdata.loc[(rowdata['Policy Nr'] == policyno) & (rowdata['time'] == '2020-02-01')]['Premium']
    if len(premium.values) > 0:
        datalive_update.loc[datalive_update['Policy Number'] == policyno, 8] = premium.values[0]
        
    premium = rowdata.loc[(rowdata['Policy Nr'] == policyno) & (rowdata['time'] == '2020-03-01')]['Premium']
    if len(premium.values) > 0:
        datalive_update.loc[datalive_update['Policy Number'] == policyno, 9] = premium.values[0]
        
    premium = rowdata.loc[(rowdata['Policy Nr'] == policyno) & (rowdata['time'] == '2020-04-01')]['Premium']
    if len(premium.values) > 0:
        datalive_update.loc[datalive_update['Policy Number'] == policyno, 10] = premium.values[0]
        


In [7]:
for index in datalive_update.index:
    try:        
        payments = datalive_update.iloc[index, 4:11].tolist()
        numberofpayments = [float(payment) for payment in payments if (isinstance(payment, float) or isinstance(payment, int)) and str(payment) != 'nan']
        paymentamounts = list(set(numberofpayments))
        # for each policy in datalive, if payments >= 4 and all payments are the same then set Freq to M
        if len(numberofpayments) > 3 and len(paymentamounts) == 1 and datalive_update.loc[index, 'Freq'] != 'M':
            datalive_update.loc[index, 'Freq'] = 'M'            
        # for each policy in datalive, if payments >=4 but all payments are not the same then mark 'Check SP' as 'Yes'
        elif len(numberofpayments) > 3 and len(paymentamounts) > 1:
            datalive_update.loc[index, 'Check SP'] = 'Yes'
        else:
            pass
    except:
        print('Err: ', datalive_update.loc[index, 'Policy Number'])
        
# for each row in datalive 'Check SP' as 'Yes', highlight the row as yellow
# df.style.apply(lambda x: ['background: lightblue' if x.name in [1,3] else '' for i in x], axis=1)
#datalive_update.style.apply(lambda x: ['background: lightgreen' if x.name in [1] else '' for i in x], axis=1).to_excel('datalive_update.xlsx', sheet_name = 'Data live', index = False)
# datalive_update.style.apply(lambda x: ['background-color: yellow' if x == 'Check' else '' for x in datalive_update['Check SP']], axis = 0)

In [8]:
datalive_update.head(10)

,Policy Number,Freq,Start,In Force,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00,...,Unnamed: 22,APE Category,Check SP,4,5,6,10,7,9,8
0,P 11593630,M,2019-10-25,8.0,-200.0,-200.0,NaN,NaN,-200.0,-200.0,...,0.0,M,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P 11593647,Q,2019-10-25,8.0,-10000.0,NaN,NaN,-10000.0,NaN,NaN,...,0.0,Q,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P 11593655,ASP,2019-10-25,8.0,-5000.0,NaN,NaN,-12000.0,NaN,NaN,...,0.0,ASP,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P 11593656,ASP,2019-10-25,8.0,-92000.0,NaN,-12000,NaN,NaN,NaN,...,0.0,ASP,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P 11593662,M,2019-10-25,8.0,-100.0,-100.0,-100,-100.0,-100.0,-100.0,...,0.0,M,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,P 11593703,ASP,2019-10-25,8.0,-75000.0,-20000.0,NaN,NaN,-40000.0,NaN,...,0.0,ASP,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,P 11593704,Q,2019-10-25,8.0,-300.0,NaN,NaN,-300.0,NaN,NaN,...,0.0,Q,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,P 11593710,M,2019-10-25,8.0,-100.0,-100.0,-100,-100.0,-100.0,-100.0,...,0.0,M,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,P 11593712,ASP,2019-10-25,8.0,-10000.0,-100.0,-200,-200.0,-200.0,-200.0,...,0.0,ASP,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,P 11593714,M,2019-10-25,8.0,-100.0,-100.0,NaN,-100.0,-100.0,NaN,...,0.0,M,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
